In [1]:
import numpy as np
import pandas as pd

In [2]:
def getExcelFrame(**kwargs):
    return pd.read_clipboard(**dict(kwargs, header = kwargs.get("header", None))).as_matrix()

In [3]:
def pasteExcelFrame(x, header = False, index = False, **kwargs):
    pd.DataFrame(x, index=None, columns=None).to_clipboard(header, index, **kwargs)

In [4]:
def getLHNormals(intervals, dimensions):
    U = np.array([(np.random.permutation(intervals) + np.random.uniform(size = intervals))/intervals for _ in range(dimensions) ]).T

In [5]:
delta_t = 1.0/12.0
r = 0.046180743016
forwards = np.array([0.00110193548246, 0.00144247473108, 0.00145756623220, 0.00164040444274, 0.00170003097283, 
0.00189334514358, 0.00207796465975, 0.00215112839404, 0.00237466736717, 0.00254524945417, 
0.00277131471170, 0.00293452717409, 0.00403240417093, 0.00452100221057, 0.00437513684684, 
0.00452100221057, 0.00437513684684, 0.00452100221057, 0.00452100221057, 0.00437513684684, 
0.00452100221057, 0.00437513684684, 0.00452100221057, 0.00452100221057, 0.00707517763454, 
0.00795618166052, 0.00769944832080, 0.00795618166052, 0.00769944832080, 0.00795618166051, 
0.00795618166052, 0.00769944832080, 0.00795618166052, 0.00769944832080, 0.00795618166052, 
0.00795618166052, 0.01092891597397, 0.01211309315752, 0.01172215743120, 0.01211309315752, 
0.01172215743120, 0.01211309315752, 0.01172215743120, 0.01211309315752, 0.01172215743120, 
0.01211309315752, 0.01211309315752, 0.01477329173649, 0.01651444153058, 0.01598136303463, 
0.01651444153058, 0.01598136303463, 0.01651444153058, 0.01651444153058, 0.01598136303463, 
0.01651444153058, 0.01598136303463, 0.01651444153058, 0.01651444153058, 0.01651444153058])

sigmas = np.array([0.3000000000000, 0.2989991638784, 0.2979974832108, 0.2949796603158, 0.2929658683192, 
0.2909484490421, 0.2889273265027, 0.2869024224366, 0.2848736562057, 0.2828409447021, 
0.2808042022478, 0.2787633404879, 0.2767182682802, 0.2746688915767, 0.2726151133008, 
0.2705568332162, 0.2684939477903, 0.2664263500482, 0.2643539294204, 0.2622765715805, 
0.2601941582742, 0.2581065671385, 0.2560136715099, 0.2539153402219, 0.2518114373892, 
0.2497018221800, 0.2475863485736, 0.2454648651029, 0.2433372145809, 0.2412032338092, 
0.2390627532678, 0.2369155967850, 0.2347615811840, 0.2326005159066, 0.2304322026107, 
0.2282564347395, 0.2260729970607, 0.2238816651716, 0.2216822049692, 0.2194743720802, 
0.2172579112484, 0.2150325556747, 0.2127980263066, 0.2105540310704, 0.2083002640421, 
0.2060364045503, 0.2037621162042, 0.2014770458390, 0.1991808223700, 0.1968730555460, 
0.1945533345898, 0.1922212267155, 0.1898762755059, 0.1875179991361, 0.1851458884232, 
0.1827594046828, 0.1803579773672, 0.1779410014583, 0.1755078345830])


In [6]:
class Curve:
    def __init__(self, discount_factors):
        self.discount_factors = discount_factors
        
    def discount_factor(self, n):
        return df[n]

In [7]:
class RandomGenerator:
    def get(self, n):
        return np.random.normal()
    
class RandomGeneratorUncorrelated:
    def get(self, n):
        return np.random.normal(size = n)
    
class PseudoRandomGenerator:
    def get(self, n):
        return 0.125

In [8]:
class CurveGenerator:  
    time_step = 0
    def __init__(self, forwards, sigmas, random_generator):
        self.init_forwards = forwards
        self.sigmas = sigmas
        self.random_generator = random_generator
        
    def getCurve(self):
        if (self.time_step == 0):
            self.forwards = self.init_forwards
        else:
            n = 60 - self.time_step
            random = self.random_generator.get(n)
            s = sigmas[:(60 - self.time_step)]
            self.forwards = self.forwards[1:]*np.exp(-(s**2/2*delta_t) + s*np.sqrt(delta_t)*random)
        
        self.time_step += 1
        discount_factors = 1.0/np.cumprod(1+delta_t*self.forwards)
        return discount_factors
    
    def getOriginalCurve(self):
        return 1.0/np.cumprod(1+delta_t*self.init_forwards)
    
    def reset(self):
        self.time_step = 0

In [9]:
class Swap:
    time_step = 0
    def __init__(self, strike):
        self.fixed_flows = np.zeros(60)
        fixed_indexes = [2 + 3*x for x in range(20)]
        self.fixed_flows[fixed_indexes] = strike        
        
        self.fixing_libor = 0.0
    
    def valuate(self, curve):
        # Fixed Leg
        fixed_value = np.dot(curve, self.fixed_flows[self.time_step:])*3.0*delta_t
        
        # Floating
        index = self.time_step % 3
        floating_value = 0.0
        if (index == 0): # If in this time_step we are fixing a libor...
            self.fixing_libor = (1.0/curve[2] - 1.0)/(3.0*delta_t)
            floating_value = 1.0 - curve[-1]
        else:
            #print "time_step: ", self.time_step, ", len(curve): ", len(curve)
            floating_value = self.fixing_libor * delta_t * 3.0 * curve[2 - index]
            floating_value += curve[2 - index] - curve[-1]
        
        self.time_step += 1
        return floating_value - fixed_value
    
    @staticmethod
    def getForward(curve):
        fixed_indexes = [2 + 3*x for x in range(20)]
        floating = 1.0 - curve[-1]
        fixing = curve[fixed_indexes].sum()*delta_t*3
        return floating/fixing
    
    def reset(self):
        self.time_step = 0
        self.fixing_libor = 0.0
        

In [38]:
class FRA:
    time_step = 0    
    
    def __init__(self, strike):
        self.strike = strike
    
    def valuate(self, curve):
        if (self.time_step > 23):
            return 0.0
        libor = (curve[23 - self.time_step]/curve[35 - self.time_step] - 1.0)/(12*delta_t)
        
        self.time_step +=1
        return 0.5*(libor - self.strike)*curve[35 - self.time_step]
    
    @staticmethod
    def getForward(curve):
        return (curve[23]/curve[35] - 1)/(delta_t*12)
    
    def reset(self):
        self.time_step = 0
    

In [11]:
class Results:
    
    def __init__(self, name, swap_npv, fra_npv):
        self.swap_npv = swap_npv
        self.fra_npv = fra_npv
        
        # Exposures
        self.swap_exposure = np.maximum(self.swap_npv, 0)
        self.fra_exposure = np.maximum(self.fra_npv, 0)

        # Porfolio exposure
        self.exposure = self.swap_exposure + self.fra_exposure
        self.exposure_netting = np.maximum(self.swap_npv + self.fra_npv, 0)
        collateral = np.concatenate((np.zeros((N,1)), self.swap_npv[:,:59] + self.fra_npv[:,:59]), axis = 1)
        self.exposure_netting_collateral = np.maximum(self.swap_npv + self.fra_npv - collateral, 0)
        
        q = 95
        self.quantiles = np.stack((np.percentile(self.swap_exposure, q, axis = 0), np.percentile(self.fra_exposure, q, axis = 0), 
                     np.percentile(self.exposure, q, axis = 0), np.percentile(self.exposure_netting, q, axis = 0),
                     np.percentile(self.exposure_netting_collateral, q, axis = 0)))
        self.means = np.stack((self.swap_exposure.mean(0), self.fra_exposure.mean(0), self.exposure.mean(0), 
                 self.exposure_netting.mean(0), self.exposure_netting_collateral.mean(0)))
        self.stdev = np.stack((self.swap_exposure.std(0), self.fra_exposure.std(0), self.exposure.std(0), 
                 self.exposure_netting.std(0), self.exposure_netting_collateral.std(0)))
        
    def printAnalysis(self):
        
        self.table = pd.DataFrame(data = {"mean": self.means.max(1), "stdev": self.stdev.max(1), "quantile 95%": self.quantiles.max(1)}, 
                             index = ["Swap", "FRA", "Porfolio", "Netting", "Collateral"])
        
        self.table.to_clipboard()
        
        print self.table

In [13]:
curve_generator = CurveGenerator(forwards, sigmas, PseudoRandomGenerator())
swap_forward = Swap.getForward(curve_generator.getOriginalCurve())
fra_forward = FRA.getForward(curve_generator.getOriginalCurve())

print "Swap Forward: ", swap_forward, ", FRA Forward: ", fra_forward

swap = Swap(swap_forward)
fra = FRA(fra_forward)

fra_npv = np.zeros(60)
swap_npv = np.zeros(60)
for time_step in range(60):
    curve = curve_generator.getCurve()
    swap_npv[time_step] = swap.valuate(curve)
    fra_npv[time_step] = fra.valuate(curve)

Swap Forward:  0.00847830283093 , FRA Forward:  0.00782510963483


In [36]:
curve_generator = CurveGenerator(forwards, sigmas, RandomGenerator())
swap_forward = Swap.getForward(curve_generator.getOriginalCurve())
fra_forward = FRA.getForward(curve_generator.getOriginalCurve())

print "Swap Forward: ", swap_forward, "FRA Forward: ", fra_forward

swap = Swap(swap_forward)
fra = FRA(fra_forward)

N = 100000
swap_npv = np.empty((N, 60))
fra_npv = np.empty((N, 60))
for path in range(N):
    curve_generator.reset()
    swap.reset()
    fra.reset()
    for time_stemp in range(60):
        curve = curve_generator.getCurve()
        swap_npv[path, time_stemp] = swap.valuate(curve)
        fra_npv[path, time_stemp] = fra.valuate(curve)
        
results_correlated = Results("Correlated", swap_npv, fra_npv)

Swap Forward:  0.00847830283093 FRA Forward:  0.00782510963483


In [37]:
curve_generator = CurveGenerator(forwards, sigmas, RandomGeneratorUncorrelated())
swap_forward = Swap.getForward(curve_generator.getOriginalCurve())
fra_forward = FRA.getForward(curve_generator.getOriginalCurve())

print "Swap Forward: ", swap_forward, "FRA Forward: ", fra_forward

swap = Swap(swap_forward)
fra = FRA(fra_forward)

N = 100000
swap_npv = np.empty((N, 60))
fra_npv = np.empty((N, 60))
for path in range(N):
    curve_generator.reset()
    swap.reset()
    fra.reset()
    for time_stemp in range(60):
        curve = curve_generator.getCurve()
        swap_npv[path, time_stemp] = swap.valuate(curve)
        fra_npv[path, time_stemp] = fra.valuate(curve)
        
results_uncorrelated = Results("Uncorrelated", swap_npv, fra_npv)

Swap Forward:  0.00847830283093 FRA Forward:  0.00782510963483


In [236]:
# Exposures
swap_exposure = np.maximum(swap_npv, 0)
fra_exposure = np.maximum(fra_npv, 0)

# Porfolio exposure
exposure = swap_exposure + fra_exposure
exposure_netting = np.maximum(swap_npv + fra_npv, 0)
collateral = np.concatenate((np.zeros((N,1)), swap_npv[:,:59] + fra_npv[:,:59]), axis = 1)
exposure_netting_collateral = np.maximum(swap_npv + fra_npv - collateral, 0)

In [237]:
# Percentile
q = 95
q_swap_exposure = np.percentile(swap_exposure, q, axis = 0)
q_fra_exposure = np.percentile(fra_exposure, q, axis = 0)
q_exposure = np.percentile(exposure, q, axis = 0)
q_exposure_netting = np.percentile(exposure_netting, q, axis = 0)
q_exposure_netting_collateral = np.percentile(exposure_netting_collateral, q, axis = 0)

In [51]:
print "############################# CORRELATED ###########################"
print "q_swap_exposure: ", np.max(q_swap_exposure)
print "q_fra_exposure: ", np.max(q_fra_exposure)
print "q_exposure: ", np.max(q_exposure)
print "q_exposure_netting: ", np.max(q_exposure_netting)
print "q_exposure_netting_collateral: ", np.max(q_exposure_netting_collateral)

############################# CORRELATED ###########################
q_swap_exposure: 

NameError: name 'q_swap_exposure' is not defined

In [234]:
print "############################# INCORRELATED ###########################"
print "q_swap_exposure: ", np.max(q_swap_exposure)
print "q_fra_exposure: ", np.max(q_fra_exposure)
print "q_exposure: ", np.max(q_exposure)
print "q_exposure_netting: ", np.max(q_exposure_netting)
print "q_exposure_netting_collateral: ", np.max(q_exposure_netting_collateral)

############################# INCORRELATED ###########################
q_swap_exposure:  0.0158083599967
q_fra_exposure:  0.000746520784434
q_exposure:  0.0158083599967
q_exposure_netting:  0.0158083599967
q_exposure_netting_collateral:  0.00249087860016


In [79]:
import matplotlib.pyplot as plt

plt.close('all')
fig = plt.figure()

#plt.plot(range(60), q_swap_exposure, 'r')
#plt.plot(range(60), q_swap_exposure, 'y')
#plt.plot(range(60), q_exposure, 'b')
#plt.plot(range(60), q_exposure_netting, 'g')
plt.plot(range(60), results_correlated.means[0], 'r')
plt.plot(range(60), results_uncorrelated.means[0], 'b')
plt.xlabel('Months')
plt.ylabel('Mean')
#plt.axis([0, 1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 59])

plt.show()

In [15]:
#results_correlated.printAnalysis()
results_uncorrelated.printAnalysis()

                mean  quantile 95%     stdev
Swap        0.011339      0.015796  0.002595
FRA         0.000170      0.000745  0.000266
Porfolio    0.011339      0.015796  0.002595
Netting     0.011339      0.015796  0.002595
Collateral  0.001785      0.002484  0.000637


In [17]:
np.percentile(results_correlated.fra_exposure + results_correlated.swap_exposure, 95, axis = 0)

array([ 0.        ,  0.00484129,  0.00700425,  0.01043959,  0.0118633 ,
        0.0132014 ,  0.01601334,  0.01713332,  0.01821529,  0.0206244 ,
        0.02166817,  0.02262528,  0.02473647,  0.02568776,  0.02648214,
        0.02787384,  0.02864963,  0.02951052,  0.03058617,  0.03139412,
        0.03216491,  0.03321135,  0.03399517,  0.03469824,  0.03279741,
        0.03352943,  0.03407402,  0.03331935,  0.03393149,  0.03437482,
        0.03359645,  0.03421405,  0.03458743,  0.03362775,  0.03408068,
        0.03467983,  0.03355575,  0.03413426,  0.03456743,  0.03180724,
        0.03219839,  0.03251293,  0.02933016,  0.02970175,  0.02997196,
        0.02682217,  0.02699795,  0.02741766,  0.02341737,  0.02370868,
        0.02392964,  0.01811614,  0.01824686,  0.01832679,  0.01237972,
        0.01241932,  0.01253399,  0.00647295,  0.00648909,  0.00650892])

In [18]:
np.percentile(np.maximum(results_correlated.fra_npv + results_correlated.swap_npv, 0.0), 95, axis = 0)

array([ 0.        ,  0.00484129,  0.00700425,  0.01043959,  0.0118633 ,
        0.0132014 ,  0.01601334,  0.01713332,  0.01821529,  0.0206244 ,
        0.02166817,  0.02262528,  0.02473647,  0.02568776,  0.02648214,
        0.02787384,  0.02864963,  0.02951052,  0.03058617,  0.03139412,
        0.03216491,  0.03321135,  0.03399517,  0.03469824,  0.03279741,
        0.03352943,  0.03407402,  0.03331935,  0.03393149,  0.03437482,
        0.03359645,  0.03421405,  0.03458743,  0.03362775,  0.03408068,
        0.03467983,  0.03355575,  0.03413426,  0.03456743,  0.03180724,
        0.03219839,  0.03251293,  0.02933016,  0.02970175,  0.02997196,
        0.02682217,  0.02699795,  0.02741766,  0.02341737,  0.02370868,
        0.02392964,  0.01811614,  0.01824686,  0.01832679,  0.01237972,
        0.01241932,  0.01253399,  0.00647295,  0.00648909,  0.00650892])

In [66]:
np.percentile(results_correlated.swap_exposure, 95, axis = 0)

array([ 0.        ,  0.00439907,  0.00632239,  0.00972112,  0.0109884 ,
        0.01212496,  0.01482283,  0.01584609,  0.01682955,  0.01920491,
        0.02015749,  0.02095815,  0.02298235,  0.02372602,  0.02444728,
        0.02579374,  0.02652147,  0.02727686,  0.02838354,  0.02897028,
        0.02973281,  0.03079387,  0.03137993,  0.03200684,  0.03284804,
        0.03351301,  0.03408816,  0.0334851 ,  0.03415529,  0.03469901,
        0.03398319,  0.03439334,  0.03487218,  0.03402762,  0.03439763,
        0.03494949,  0.03393035,  0.03433053,  0.03463089,  0.03187994,
        0.03232272,  0.03265859,  0.02953375,  0.02982331,  0.03010619,
        0.02677868,  0.02704134,  0.02738196,  0.02343008,  0.02368705,
        0.02387533,  0.01809104,  0.01820418,  0.01836964,  0.0123815 ,
        0.01245218,  0.01251763,  0.00643945,  0.00645817,  0.00647668])

In [62]:
results_uncorrelated.quantiles[0] - results_uncorrelated2.quantiles[0]

array([  0.00000000e+00,   5.25056638e-06,  -1.18001955e-06,
         4.91476538e-06,   3.69079847e-06,  -1.12485936e-05,
        -8.05617649e-06,  -1.00465760e-06,  -8.66985399e-06,
        -1.28843788e-05,  -4.20239660e-06,  -1.57645595e-05,
        -1.10156096e-05,  -1.20510956e-05,  -1.32456789e-05,
        -1.13415579e-05,  -1.29366826e-05,  -2.10944085e-05,
         1.01276078e-06,   4.07230056e-06,   3.11509947e-05,
         1.04165351e-05,   9.59547386e-06,   7.25831300e-06,
         2.14018355e-05,   1.98012878e-05,   3.72139564e-06,
         1.88566433e-05,   5.16607843e-06,   2.94759565e-05,
         9.36633359e-08,   2.96787908e-06,   1.87525995e-05,
         1.12025596e-05,   3.40392272e-05,   4.34879118e-05,
         3.56253990e-05,   4.76504251e-05,   1.75544696e-05,
         3.36111076e-05,   2.52735210e-05,   2.79849221e-05,
         3.51671165e-05,   1.09138718e-05,   3.16584073e-05,
         9.49554422e-06,   1.54950033e-05,  -6.28786513e-06,
        -1.38464746e-05,

In [211]:
fra_npv[0,:]

array([  0.00000000e+000,   9.77761733e-002,   4.77151525e-013,
         1.20684436e-049,   1.08060768e-084,   5.47222805e-115,
         1.76292209e-140,   1.33698485e-161,               inf,
                     nan,               nan,               nan,
                     nan,               nan,               nan,
                     nan,               nan,               nan,
                     nan,               nan,               nan,
                     nan,               nan,               nan,
         0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
         0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
         0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
         0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
         0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
         0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
         0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
         0.00000000e+000,   0.00000000e+

In [287]:
a[[1, 2, 3]]

array([ 0.00158306,  0.00178116,  0.00184444])

In [137]:
print(a[1:])
1.0/np.cumprod(a[1:])

[1 2 3 4 5 6 7 8 9]


array([  1.00000000e+00,   5.00000000e-01,   1.66666667e-01,
         4.16666667e-02,   8.33333333e-03,   1.38888889e-03,
         1.98412698e-04,   2.48015873e-05,   2.75573192e-06])

In [237]:
c = CurveGenerator(forwards, sigmas, PseudoRandomGenerator())

In [386]:
pasteExcelFrame(fra_npv, header=[], float_format='%.15f')

In [275]:
curva = c.getCurve()

In [274]:
for i in range(28):
    c.getCurve()

In [276]:
curva

array([ 0.99803898,  0.99608565,  0.99414006])

In [179]:
a = forwards[1:]*np.exp( -(sigmas**2/2*delta_t) + sigmas*np.sqrt(delta_t)*1)

In [189]:
a.shape

(59L,)

In [168]:
a = np.arange(5)
b = np.random.normal(size = 5)

In [175]:
a[:4]

array([0, 1, 2, 3])

In [187]:
forwards

array([ 0.00110194,  0.00144247,  0.00145757,  0.0016404 ,  0.00170003,
        0.00189335,  0.00207796,  0.00215113,  0.00237467,  0.00254525,
        0.00277131,  0.00293453,  0.0040324 ,  0.004521  ,  0.00437514,
        0.004521  ,  0.00437514,  0.004521  ,  0.004521  ,  0.00437514,
        0.004521  ,  0.00437514,  0.004521  ,  0.004521  ,  0.00707518,
        0.00795618,  0.00769945,  0.00795618,  0.00769945,  0.00795618,
        0.00795618,  0.00769945,  0.00795618,  0.00769945,  0.00795618,
        0.00795618,  0.01092892,  0.01211309,  0.01172216,  0.01211309,
        0.01172216,  0.01211309,  0.01172216,  0.01211309,  0.01172216,
        0.01211309,  0.01211309,  0.01477329,  0.01651444,  0.01598136,
        0.01651444,  0.01598136,  0.01651444,  0.01651444,  0.01598136,
        0.01651444,  0.01598136,  0.01651444,  0.01651444,  0.01651444])

In [199]:

x = np.random.normal(size=len(sigmas))
(-(sigmas**2/2*delta_t) + sigmas*np.sqrt(delta_t)*x)


array([-0.09537105,  0.07069073, -0.0247068 , -0.00338331, -0.07850311,
        0.14465246,  0.02216471, -0.08537685,  0.03405268, -0.07652605,
        0.0004663 , -0.1020206 , -0.112918  ,  0.02081393, -0.0368906 ,
       -0.02321228, -0.0679583 ,  0.01261837,  0.07274626,  0.02740244,
        0.00398133, -0.03521626,  0.06389698, -0.00474645, -0.00236629,
       -0.03367848, -0.05423343, -0.01540825,  0.01031354, -0.02263359,
       -0.02672333,  0.02586256, -0.070428  ,  0.07338364, -0.03970403,
       -0.07247747, -0.01446248, -0.01714092, -0.03903464,  0.00405507,
        0.09661802, -0.05324921, -0.02630302, -0.04537451, -0.05433551,
        0.12494054, -0.1092551 ,  0.06369234, -0.07515961, -0.07184394,
       -0.00068073,  0.05664926, -0.04481063, -0.05677608,  0.03804432,
        0.01942447,  0.00381758, -0.01462947, -0.00585805])

In [207]:
x = np.ones(59)*0.5
(-(sigmas**2/2*delta_t) + sigmas*np.sqrt(delta_t)*x)

array([ 0.03955127,  0.03943179,  0.03931213,  0.0389511 ,  0.03870977,
        0.03846767,  0.03822478,  0.03798109,  0.0377366 ,  0.03749128,
        0.03724514,  0.03699815,  0.0367503 ,  0.03650158,  0.03625198,
        0.03600147,  0.03575005,  0.03549771,  0.03524441,  0.03499015,
        0.03473492,  0.03447868,  0.03422143,  0.03396315,  0.03370381,
        0.0334434 ,  0.03318189,  0.03291926,  0.03265549,  0.03239056,
        0.03212444,  0.03185711,  0.03158854,  0.0313187 ,  0.03104757,
        0.0307751 ,  0.03050128,  0.03022608,  0.02994945,  0.02967136,
        0.02939177,  0.02911065,  0.02882796,  0.02854365,  0.02825768,
        0.02797   ,  0.02768057,  0.02738933,  0.02709623,  0.02680122,
        0.02650423,  0.0262052 ,  0.02590407,  0.02560077,  0.02529522,
        0.02498734,  0.02467706,  0.02436428,  0.02404892])

In [222]:
sigmas.shape

(59L,)